In [4]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import Tool
import requests
from bs4 import BeautifulSoup

# =========================
# Initialize Ollama LLM
# =========================
llm = OllamaLLM(model="llama3.1", temperature=0)

# =========================
# Tool 1: QA Tool
# =========================
qa_prompt = PromptTemplate.from_template("Answer clearly: {question}")
qa_chain = qa_prompt | llm
qa_tool = Tool(
    name="QA Tool",
    func=lambda q: qa_chain.invoke({"question": q}),
    description="Answer factual questions clearly"
)

# =========================
# Tool 2: Summarizer Tool
# =========================
sum_prompt = PromptTemplate.from_template("Summarize the following text:\n{text}")
sum_chain = sum_prompt | llm
sum_tool = Tool(
    name="Summarizer Tool",
    func=lambda t: sum_chain.invoke({"text": t}),
    description="Summarizes input text"
)

# =========================
# Tool 3: Web Search (Google scraping)
# =========================
def simple_search(query):
    url = f"https://www.google.com/search?q={query}"
    headers = {"User-Agent": "Mozilla/5.0"}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")
    results = []
    for g in soup.find_all('div', class_='tF2Cxc')[:3]:
        title = g.find('h3').text if g.find('h3') else ""
        link = g.find('a')['href'] if g.find('a') else ""
        results.append(f"{title} - {link}")
    return "\n".join(results)

search_tool = Tool(
    name="Web Search",
    func=simple_search,
    description="Search Google without API key"
)

# =========================
# Manual multi-tool execution
# =========================
tools = {
    "QA": qa_tool,
    "Summarizer": sum_tool,
    "Web Search": search_tool
}

queries = [
    ("QA", "What is LangGraph in LangChain?"),
    ("Summarizer", "LangChain is a framework to build LLM apps using prompts, memory, tools, and agents."),
    ("Web Search", "Latest updates on GPT-4")
]

for tool_name, user_input in queries:
    print(f"\n🧑‍💻 Tool: {tool_name}")
    print(f"Input: {user_input}")
    output = tools[tool_name].run(user_input)
    print(f"\n🤖 Output:\n{output}")



🧑‍💻 Tool: QA
Input: What is LangGraph in LangChain?

🤖 Output:
In LangChain, LangGraph refers to a graph data structure that represents the relationships between different entities and pieces of information within a language model's output. It's essentially a way to visualize and interact with the knowledge graph generated by the model.

Think of it like a map of concepts, entities, and their connections, which can be used for various tasks such as:

1. **Knowledge graph construction**: LangGraph helps build a structured representation of the knowledge extracted from the language model.
2. **Question answering**: It enables more accurate question-answering by allowing models to reason over the relationships between entities in the graph.
3. **Text generation**: LangGraph can be used as a source of information for generating text, such as summaries or explanations.

In summary, LangGraph is an essential component of LangChain that facilitates the creation and manipulation of knowledge 